# imports

In [2]:
import requests
import pandas as pd

# acquire data

In [8]:
api_url="https://opendata.maryland.gov/api/id/crti-ybyp.json?$select=*&$order=`:id`+ASC&$limit=3000&$offset=0"
resp = requests.get(api_url)
data = resp.json()

#clean data

In [9]:
type(data)

list

In [10]:
data[0]

{'sno': '1',
 'complaint': '232264',
 'complaint_description': 'concern of loud noise in the Cheverly area',
 'complaint_type': 'Other',
 'recieved_date': '2024-03-06',
 'incident_date': '2024-03-06',
 'county': "Prince George's",
 'incident_closed_date': '2024-03-08',
 'incident_status_desc': 'Incident Closed-Managed'}

In [11]:
df = pd.DataFrame(data)

In [12]:
backup_df=df.copy()

In [ ]:
# clean data

In [14]:
df.dtypes

sno                      object
complaint                object
complaint_description    object
complaint_type           object
recieved_date            object
incident_date            object
county                   object
incident_closed_date     object
incident_status_desc     object
incident_zip             object
dtype: object

# analyze data

In [15]:
df['recieved_date'] = pd.to_datetime(df['recieved_date'])

In [16]:
df.dtypes

sno                              object
complaint                        object
complaint_description            object
complaint_type                   object
recieved_date            datetime64[ns]
incident_date                    object
county                           object
incident_closed_date             object
incident_status_desc             object
incident_zip                     object
dtype: object

In [17]:
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['incident_closed_date'] = pd.to_datetime(df['incident_closed_date'])

In [18]:
df.dtypes

sno                              object
complaint                        object
complaint_description            object
complaint_type                   object
recieved_date            datetime64[ns]
incident_date            datetime64[ns]
county                           object
incident_closed_date     datetime64[ns]
incident_status_desc             object
incident_zip                     object
dtype: object

In [19]:
# For the summaries we're running, we'll limit that to just complaints received in the last year?

year_ago_date = pd.Timestamp('2023-03-09')

last_year_df = df[df["recieved_date"] > year_ago_date].copy()

# How many are in each county?
last_year_df["county"].value_counts()

county
Baltimore City        101
Anne Arundel           85
Prince George's        83
Frederick              62
Baltimore              57
Montgomery             35
Cecil                  25
Dorchester             17
Harford                17
Howard                 15
Allegany               15
Washington             14
Charles                12
Carroll                 9
Wicomico                8
Not Yet Determined      7
Worcester               7
St. Mary's              6
Garrett                 5
Caroline                3
Queen Anne's            2
Somerset                2
Talbot                  1
Kent                    1
Statewide               1
Name: count, dtype: int64

In [20]:
# What are the most common types of complaint?
# last_year_df["complaint_type"]
last_year_df["complaint_type"].value_counts()

complaint_type
Air                                                                                 166
Odor                                                                                140
Other                                                                               121
Fugitive Dust/Particulate Matter                                                     35
Smoke                                                                                28
Air, Fumes, Odor                                                                     20
Open Burning                                                                         13
Air, Odor                                                                            11
Fumes                                                                                11
Open Burning, Smoke                                                                   7
Air, Odor, Smoke                                                                      4
Air, Fugitive Dus

In [21]:
# Wait, though. The second line has a type of "Air, Other" — shouldn't that count toward both categories?
# The data's bunched up, so we need to make the table longer.
# (Important caveat to be thinking what each row represents — you could fall into a trap with this transformation.)
last_year_df["complaint_type"] = last_year_df["complaint_type"].str.split(', ')

In [22]:
# Look at the first five rows.
last_year_df[0:5]

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232264,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
1,2,232263,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
2,3,232262,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
3,4,232261,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
4,5,232260,concern of loud noise in the Cheverly area,[Other],2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN


In [25]:
# Now, see what it looks like when we run "explode" on these five rows.
last_year_df[0:20].explode("complaint_type")

,sno,complaint,complaint_description,complaint_type,recieved_date,incident_date,county,incident_closed_date,incident_status_desc,incident_zip
0,1,232264,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
1,2,232263,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
2,3,232262,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
3,4,232261,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
4,5,232260,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
5,6,232259,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
6,7,232258,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
7,8,232257,concern of loud noise in the Cheverly area,Other,2024-03-06,2024-03-06,Prince George's,2024-03-08,Incident Closed-Managed,NaN
8,9,232253,Complainant states odors from Stahl Point Road...,Air,2024-03-07,2024-03-07,Anne Arundel,NaT,Under Investigation,NaN
9,10,232252,Complainant states odor from the Curtis Medica...,Air,2024-03-07,2024-03-06,Anne Arundel,NaT,Under Investigation,NaN


In [26]:
# Now, apply this to our full dataset.
complaints_by_type = last_year_df.explode("complaint_type")

In [27]:
# Now we can see how many times each type of complaint was lodged.
last_year_by_type = complaints_by_type["complaint_type"].value_counts()

last_year_by_type

# We'll leave the complaints-by-type dataset there for now, but note that we _could_ export one CSV per complaint type,
# if for example that would help reporters on different beats focus on their relevant parts of the same data.

complaint_type
Air                                   223
Odor                                  186
Other                                 126
Smoke                                  56
Fugitive Dust/Particulate Matter       45
Fumes                                  39
Open Burning                           30
Asbestos Complaint                      3
Air Pollutant Release                   2
Noise Complaint                         2
Non-Tidal Wetlands/Waterway             2
Suspected Operation without Permit      1
ARA AQCP non-regulated entity           1
Name: count, dtype: int64

In [28]:
# There are some `NaN` values in the county column. Let's toss them out from the main dataframe.
# First, we'll see how many rows are missing one value or the other.
last_year_df["county"].isna().sum()

9

In [29]:
# We know we have just a handful of NAs, but what about others? Let's look at all unique county values first.
last_year_df["county"].unique()

array(["Prince George's", 'Anne Arundel', 'Wicomico', 'Dorchester',
       'Carroll', 'Frederick', 'Charles', nan, 'Montgomery', 'Worcester',
       'Baltimore', 'Cecil', 'Allegany', 'Baltimore City', 'Talbot',
       'Not Yet Determined', 'Howard', 'Caroline', 'Washington',
       "Queen Anne's", 'Harford', 'Garrett', "St. Mary's", 'Kent',
       'Statewide', 'Somerset'], dtype=object)

In [30]:
# It looks like there are three more we should screen out: "Not Yet Determined", "Outside of Maryland" and "Statewide."
# We'll create other dataframes with just those values — and export them to CSV files so that we have the option of looking through those one by one at some point.
null_county_rows = last_year_df[last_year_df['county'].isnull()]
wrong_county_rows = last_year_df[last_year_df['county'].isin(["Not Yet Determined", "Outside of Maryland", "Statewide"])]

# Export each one separately.
null_county_rows.to_csv("exported_data/null_counties.csv")
wrong_county_rows.to_csv("exported_data/wrong_counties.csv")

In [31]:
# Now, do a new dataframe with only the values we expect.
last_year_df = last_year_df[last_year_df['county'].isnull() == False]
last_year_df = last_year_df[last_year_df['county'].isin(["Not Yet Determined", "Outside of Maryland", "Statewide"]) == False]

In [32]:
# Let's go back to the "one row = one complaint" dataset, and look at how many complaints have come in from each county.
last_year_by_county = last_year_df["county"].value_counts()

last_year_by_county

county
Baltimore City     101
Anne Arundel        85
Prince George's     83
Frederick           62
Baltimore           57
Montgomery          35
Cecil               25
Harford             17
Dorchester          17
Howard              15
Allegany            15
Washington          14
Charles             12
Carroll              9
Wicomico             8
Worcester            7
St. Mary's           6
Garrett              5
Caroline             3
Queen Anne's         2
Somerset             2
Talbot               1
Kent                 1
Name: count, dtype: int64

# export data

In [33]:
# Let's write out our main dataset as a CSV.
df.to_csv(
    f"exported_data/all_complaints.csv",
    index=False,
    columns=[
        "complaint",
        "incident_date",
        "county",
        "incident_zip",
        "complaint_type",
        "complaint_description",
        "recieved_date",
        "incident_status_desc",
        "incident_closed_date",
    ]
)

In [34]:
# We've also effectively made a couple of pivot tables (reports per county and frequency of report type). Let's export them, too.
last_year_by_type.to_csv("exported_data/complaint_typef_frequency.csv")
last_year_by_county.to_csv("exported_data/county_frequency.csv")